# Importe modules

In [1]:
# Computational modules 
%matplotlib inline
import xarray as xr
import glob
import os
import numpy as np
import netCDF4
from netCDF4 import Dataset
import pandas as pd
import re
from array import array
from pylab import *

# Plotting modules 
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas.plotting
import matplotlib.ticker as ticker

# Read amip-piForcing : radiative budget, tas

In [25]:
#------------- open files and read variables ----------------------
path = "/scratch/globc/peatier/CNRM-CM6-1/CFMIP/amip"
variables=['rlut', 'rsut', 'rsdt', 'tas']
filename_ctl_rlut = "/"+variables[0]+"_Amon_CNRM-CM6-1_amip-piForcing_r1i1p1f2_gr_187001-201412.nc"
filename_ctl_rsut = "/"+variables[1]+"_Amon_CNRM-CM6-1_amip-piForcing_r1i1p1f2_gr_187001-201412.nc"
filename_ctl_rsdt = "/"+variables[2]+"_Amon_CNRM-CM6-1_amip-piForcing_r1i1p1f2_gr_187001-201412.nc"
filename_ctl_tas = "/"+variables[3]+"_Amon_CNRM-CM6-1_amip-piForcing_r1i1p1f2_gr_187001-201412.nc"

ctl_rlut = xr.open_dataset(path+filename_ctl_rlut)    
ctl_rsut = xr.open_dataset(path+filename_ctl_rsut)  
ctl_rsdt = xr.open_dataset(path+filename_ctl_rsdt)  
ctl_tas = xr.open_dataset(path+filename_ctl_tas)  

#------------- rlut ----------------------
df_ctl_rlut=ctl_rlut['rlut'].to_dataframe()
df_ctl_rlut_year=df_ctl_rlut.reset_index(level=['time'])
df_ctl_rlut_year['year'] = pd.DatetimeIndex(df_ctl_rlut_year['time']).year
df_ctl_rlut_clim=df_ctl_rlut_year.groupby(['year']).mean()
#df_ctl_rlut_clim

#------------- rsut ----------------------
df_ctl_rsut=ctl_rsut['rsut'].to_dataframe()
df_ctl_rsut_year=df_ctl_rsut.reset_index(level=['time'])
df_ctl_rsut_year['year'] = pd.DatetimeIndex(df_ctl_rsut_year['time']).year
df_ctl_rsut_clim=df_ctl_rsut_year.groupby(['year']).mean()
#df_ctl_rsut_clim

#------------- rsdt ----------------------
df_ctl_rsdt=ctl_rsdt['rsdt'].to_dataframe()
df_ctl_rsdt_year=df_ctl_rsdt.reset_index(level=['time'])
df_ctl_rsdt_year['year'] = pd.DatetimeIndex(df_ctl_rsdt_year['time']).year
df_ctl_rsdt_clim=df_ctl_rsdt_year.groupby(['year']).mean()
#df_ctl_rsdt_clim

#------------- tas ----------------------
df_ctl_tas=ctl_tas['tas'].to_dataframe().drop('height',axis=1)
df_ctl_tas_year=df_ctl_tas.reset_index(level=['time'])
df_ctl_tas_year['year'] = pd.DatetimeIndex(df_ctl_tas_year['time']).year
df_ctl_tas_clim=df_ctl_tas_year.groupby(['year']).mean()
#df_ctl_tas_clim

#------------ Merge -------------------
ctl1=pd.merge(df_ctl_rlut_clim, df_ctl_rsut_clim,left_index=True,right_index=True)
ctl2=pd.merge(ctl1, df_ctl_rsdt_clim,left_index=True,right_index=True)
ctl=pd.merge(ctl2, df_ctl_tas_clim,left_index=True,right_index=True)
ctl

,rlut,rsut,rsdt,tas
year,,,,
1870,223.725830,103.911697,298.951080,277.275330
1871,223.738297,103.664124,298.951904,277.342285
1872,223.750626,103.670815,299.023926,277.384857
1873,223.844925,103.754616,298.950226,277.331146
1874,223.465515,103.960541,298.951080,277.144165
...,...,...,...,...
2010,225.047501,103.597878,298.951080,278.120728
2011,224.794678,103.102051,298.951904,278.024933
2012,225.122086,103.167130,299.023926,278.249817


# Read amip-p4K : radiative budget, tas

In [27]:
#------------- open files and read variables ----------------------
filename_p4K_rlut = "/"+variables[0]+"_Amon_CNRM-CM6-1_amip-p4K_r1i1p1f2_gr_197901-201412.nc"
filename_p4K_rsut = "/"+variables[1]+"_Amon_CNRM-CM6-1_amip-p4K_r1i1p1f2_gr_197901-201412.nc"
filename_p4K_rsdt = "/"+variables[2]+"_Amon_CNRM-CM6-1_amip-p4K_r1i1p1f2_gr_197901-201412.nc"
filename_p4K_tas = "/"+variables[3]+"_Amon_CNRM-CM6-1_amip-p4K_r1i1p1f2_gr_197901-201412.nc"

p4K_rlut = xr.open_dataset(path+filename_p4K_rlut)    
p4K_rsut = xr.open_dataset(path+filename_p4K_rsut)  
p4K_rsdt = xr.open_dataset(path+filename_p4K_rsdt)  
p4K_tas = xr.open_dataset(path+filename_p4K_tas)  

#------------- rlut ----------------------
df_p4K_rlut=ctl_rlut['rlut'].to_dataframe()
df_p4K_rlut_year=df_p4K_rlut.reset_index(level=['time'])
df_p4K_rlut_year['year'] = pd.DatetimeIndex(df_p4K_rlut_year['time']).year
df_p4K_rlut_clim=df_ctl_rlut_year.groupby(['year']).mean()
#df_p4K_rlut_clim

#------------- rsut ----------------------
df_p4K_rsut=ctl_rsut['rsut'].to_dataframe()
df_p4K_rsut_year=df_p4K_rsut.reset_index(level=['time'])
df_p4K_rsut_year['year'] = pd.DatetimeIndex(df_p4K_rsut_year['time']).year
df_p4K_rsut_clim=df_p4K_rsut_year.groupby(['year']).mean()
#df_p4K_rsut_clim

#------------- rsdt ----------------------
df_p4K_rsdt=ctl_rsdt['rsdt'].to_dataframe()
df_p4K_rsdt_year=df_p4K_rsdt.reset_index(level=['time'])
df_p4K_rsdt_year['year'] = pd.DatetimeIndex(df_p4K_rsdt_year['time']).year
df_p4K_rsdt_clim=df_p4K_rsdt_year.groupby(['year']).mean()
#df_p4K_rsdt_clim

#------------- tas ----------------------
df_p4K_tas=ctl_tas['tas'].to_dataframe().drop('height',axis=1)
df_p4K_tas_year=df_p4K_tas.reset_index(level=['time'])
df_p4K_tas_year['year'] = pd.DatetimeIndex(df_p4K_tas_year['time']).year
df_p4K_tas_clim=df_p4K_tas_year.groupby(['year']).mean()
#df_p4K_tas_clim

#------------ Merge -------------------
p4K1=pd.merge(df_p4K_rlut_clim, df_p4K_rsut_clim,left_index=True,right_index=True)
p4K2=pd.merge(p4K1, df_p4K_rsdt_clim,left_index=True,right_index=True)
p4K=pd.merge(p4K2, df_p4K_tas_clim,left_index=True,right_index=True)
p4K

,rlut,rsut,rsdt,tas
year,,,,
1870,223.725830,103.911697,298.951080,277.275330
1871,223.738297,103.664124,298.951904,277.342285
1872,223.750626,103.670815,299.023926,277.384857
1873,223.844925,103.754616,298.950226,277.331146
1874,223.465515,103.960541,298.951080,277.144165
...,...,...,...,...
2010,225.047501,103.597878,298.951080,278.120728
2011,224.794678,103.102051,298.951904,278.024933
2012,225.122086,103.167130,299.023926,278.249817
